In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager # chrome브라우저 버전에 맞는 드라이버인지 확인 및 없으면 다운로드
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from dateutil import parser
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
import time, json, logging, requests, os
from bs4 import BeautifulSoup
import boto3

logging.basicConfig(level=logging.INFO)  # 로그 레벨 설정
logger = logging.getLogger(__name__)

In [2]:
car = {'산타페': [ # 차종
            '산타페', # 해당 차종의 이명
            '싼타페']
       }
BASE_URL = "https://gall.dcinside.com/board/lists/?id=car_new1"

# 차량 출시일 / +3개월
SEARCH_START = "2023-08-16"
SEARCH_END = "2023-11-16"

# 제목만 / 제목+내용
SEARCH_URL_TITLE = f"https://gall.dcinside.com/board/lists/?id=car_new1&s_type=search_subject&s_keyword={car['산타페'][1]}"
SEARCH_URL_TITLE_AND_CONTENT = f"https://gall.dcinside.com/board/lists/?id=car_new1&s_type=search_subject_memo&s_keyword={car['산타페'][1]}"


In [36]:
class DC_crawler:
    MAX_TRY = 2
    RETRY_WAITS = 2
    post_link = [
    ]
    
    def __init__(self, s_date, e_date, car_id):
        self.start_date = s_date
        self.end_date = e_date
        self.car_id = car_id
    
    # Chrome WebDriver 선언
    def _get_driver(self,):
        # 이 path는 로컬 실행 시 주석처리 하세요.
        # chrome_path = "/opt/chrome/chrome-headless-shell-mac-arm64"
        # driver_path = "/opt/chromedriver"   

        options = webdriver.ChromeOptions()
        # options.binary_location = chrome_path  # Chrome 실행 파일 지정 (로컬 실행 시 주석 처리)
        options.add_argument("--headless")  # Headless 모드
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-gpu")
        options.add_argument("user-agent=Mozilla/5.0 (compatible; Daum/3.0; +http://cs.daum.net/)")
        options.add_argument("--window-size=1920x1080")
        
        # service = Service(executable_path="/opt/chromedriver")
        driver = webdriver.Chrome(
            # service=service, # 로컬 실행 시 주석 처리
            options=options) 
        return driver
    
    def get_entry_point(self, driver:webdriver.Chrome, url=SEARCH_URL_TITLE):
        s_date = self.start_date
        e_date = self.end_date
        
        driver.get(url)
        
        #-----------------------------------------------
        # 🔹 1. 날짜 검색 창을 여는 버튼 클릭
        #-----------------------------------------------
        open_date_picker = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn_grey_roundbg.btn_schmove")))
        open_date_picker.click()
        time.sleep(1)  # 검색 창이 뜨는 시간 고려
        
        #-----------------------------------------------
        # 🔹 2. 날짜 입력 필드 찾기
        #-----------------------------------------------
        date_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "input.dayin.calendar")))
        
        #-----------------------------------------------
        # 🔹 3. 날짜 입력
        #-----------------------------------------------
        target_date = e_date  # 검색할 날짜
        # JavaScript로 날짜 값 변경
        driver.execute_script("arguments[0].value = arguments[1];", date_input, target_date)
        date_input.send_keys(target_date)
        date_input.send_keys(Keys.RETURN)  # 엔터 입력

        #-----------------------------------------------
        # 🔹 4. 검색 버튼 클릭
        #-----------------------------------------------
        search_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn_blue.small.fast_move_btn"))
        )
        search_btn.click()

        #-----------------------------------------------
        # 🔹 5. 검색 결과 로딩 대기
        #-----------------------------------------------
        time.sleep(0.5)  # 네트워크 환경에 따라 조정
        
        #-----------------------------------------------
        # 🔹 6. 페이지 소스 가져오기
        #-----------------------------------------------
        current_page_url = driver.current_url
        return current_page_url        
        
        
    def crawl_post_link(self, driver:webdriver.Chrome, soup:BeautifulSoup):
        """
        현재 페이지에서 게시글들의 링크를 수집합니다.
        """
        posts = soup.select("tr.ub-content.us-post")
        
        date_changed = None
        for post in posts:
            # 날짜 검증
            date = post.select_one("td.gall_date").get_text(strip=True) if post.select_one("td_gall_date") else "날짜 없음"

            gall_num = int(post.select_one("td.gall_num").get_text(strip=True))
            dc_url = "https://gall.dcinside.com"
            title_tag = post.select_one("td.gall_tit.ub-word a")
            link = dc_url + title_tag["href"] if title_tag else "링크 없음"
            
            post_info = {
                "url" : link,
                "id" : gall_num,
                "date" : date
            }
            
            self.post_link.append(post_info)
        return True
    
    def page_traveler(self, driver:webdriver.Chrome, current_link:str):
        """
        페이징 박스를 순회합니다.
        시간 역순으로 순회합니다. 
        (페이징 박스는 정방향 순회, 보이는 게시글은 시간 역순)
        """
        random_sleep_time = [0.8, 0.6, 0.7, 0.5]
        i = 0
        # for _ in range(5):
        driver.get(current_link)
        web_source = driver.page_source
        soup = BeautifulSoup(web_source, "html.parser")
        return soup
            if self.crawl_post_link(driver, soup): # 유효하지 않은 날짜를 만날 때 까지 크롤링
                # 한 페이지를 다 긁었으면...
                current_page = soup.find('em')
                dc_url = "https://gall.dcinside.com"
                next_link = current_page.find_next_sibling('a')
                current_link = dc_url + next_link['href']
                
                if next_link['class'] == "search_next": 
                    logger.info("Search next 10000 posts")
                
                time.sleep(random_sleep_time[i := i % 4])
                i += 1
            
            else: # 특정 범위의 날짜를 전부 크롤링 했다면
                logger.info(f"crawling {self.start_date} ~ {self.end_date} finished")
                break
        return
    


IndentationError: unexpected indent (3290091979.py, line 120)

In [37]:
logger.info("Initiating Crawler")
crawler = DC_crawler(s_date="2023-08-16", e_date="2023-11-16", car_id="santafe")
    

INFO:__main__:Initiating Crawler


In [38]:
driver = crawler._get_driver()

In [39]:
endpoint = crawler.get_entry_point(driver)

In [40]:
endpoint

'https://gall.dcinside.com/board/lists/?id=car_new1&search_pos=-8697091&s_type=search_subject&s_keyword=.EC.8B.BC.ED.83.80.ED.8E.98'

In [41]:
soup = crawler.page_traveler(driver, endpoint)

In [42]:
if crawler.crawl_post_link(driver, soup): # 유효하지 않은 날짜를 만날 때 까지 크롤링
    # 한 페이지를 다 긁었으면...
    current_page = soup.find('em')
    dc_url = "https://gall.dcinside.com"
    next_link = current_page.find_next_sibling('a')
    current_link = dc_url + next_link['href']
    
    if next_link['class'] == "search_next": 
        logger.info("Search next 10000 posts")
    
    # time.sleep(random_sleep_time[i := i % 4])
    i += 1

TypeError: 'NoneType' object is not subscriptable

In [46]:
soup
current_page = soup.select_one('.bottom_paging_box.iconpaging em')


In [47]:
current_page

<em>1</em>

In [49]:
dc_url = "https://gall.dcinside.com"
next_link = current_page.find_next_sibling('a')
current_link = dc_url + next_link['href']

if next_link.__class__ == "search_next": 
    logger.info("Search next 10000 posts")